# Torch
## Check GPU¶

In [1]:
#from apex import amp,optimizers

In [1]:
import torch
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(1))

TITAN Xp


## Set torch default parameters¶

In [2]:
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=8)
torch.backends.cudnn.benchmark = True

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/train_cnn_gp_torch')

# Set Arguments

In [3]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()

'''Training Parameters'''
parser.add_argument('--batch_size', type=int, default=300, help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200, help='number of epochs')
parser.add_argument('--grad_clip', type=float, default=5., help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.005, help='learning rate')
parser.add_argument('--learning_rate_clip', type=float, default=0.0000001, help='learning rate clip')
parser.add_argument('--decay_rate', type=float, default=.98, help='decay rate for rmsprop')
parser.add_argument('--weight_decay', type=float, default=.0001, help='decay rate for rmsprop')
parser.add_argument('--batch_norm_decay', type=float, default=.999, help='decay rate for rmsprop')
parser.add_argument('--keep_prob', type=float, default=1.0, help='dropout keep probability')
parser.add_argument('--lamda_weights', type=float, default=.01, help='lamda weight')
parser.add_argument('--data_argumentation', type=bool, default=True, help='whether do data argument')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data nomalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')
parser.add_argument('--output_dim', default=3, type=int, help='output dimention.')
parser.add_argument('--feat_dim', default=128, type=int, help='feature dimention.')

'''Configure'''
parser.add_argument('--network', type=str, default='vggnet_localization')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/mogp_net_torch', help='rnn, gru, or lstm')
'''
parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_08',
                                                            '/notebooks/michigan_nn_data/2012_01_15',
                                                            '/notebooks/michigan_nn_data/2012_01_22',
                                                            '/notebooks/michigan_nn_data/2012_02_02',
                                                            '/notebooks/michigan_nn_data/2012_02_04',
                                                            '/notebooks/michigan_nn_data/2012_02_05',
                                                            '/notebooks/michigan_nn_data/2012_03_31',
                                                            '/notebooks/michigan_nn_data/2012_09_28'])
'''
parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])

parser.add_argument('--seed', default=1337, type=int)
parser.add_argument('--save_every', type=int, default=2000, help='save frequency')
parser.add_argument('--display', type=int, default=50, help='display frequency')

sys.argv = ['']
args = parser.parse_args()

# Load Dataset

In [4]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
from PIL import Image
import numpy as np
import random

#import gpflow.multioutput.kernels as mk
import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.utils import LocalizationDataset
import time

'''
transform = transforms.Compose([transforms.ToTensor()])
dataset = LocalizationDataset(dataset_dirs = args.train_dataset, \
                              image_size = args.target_image_size, \
                              transform = transform,
                              get_pair = False)
'''
x_train = torch.from_numpy(np.genfromtxt('/notebooks/michigan_nn_data/toy/x_train.csv', delimiter=',')).float()
y_train = torch.from_numpy(np.genfromtxt('/notebooks/michigan_nn_data/toy/y_train.csv', delimiter=',')).float()
dataset = TensorDataset(x_train,y_train)

#[args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
#torch.save([args.norm_mean, args.norm_std], '/notebooks/global_localization/norm_mean_std.pt')

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=True, pin_memory=True)

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


In [6]:
#args.norm_mean, args.norm_std

# Define Model

In [5]:
class MultitaskGPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        # We have to mark the CholeskyVariationalDistribution as batch
        # so that we learn a variational distribution for each task
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(-2), batch_shape=torch.Size([3])
        )

        # We have to wrap the VariationalStrategy in a MultitaskVariationalStrategy
        # so that the output will be a MultitaskMultivariateNormal rather than a batch output
        variational_strategy = gpytorch.variational.MultitaskVariationalStrategy(
            gpytorch.variational.VariationalStrategy(
                self, inducing_points, variational_distribution, learn_inducing_locations=True
            ), num_tasks=3
        )

        super().__init__(variational_strategy)

        # The mean and covariance modules should be marked as batch
        # so we learn a different set of hyperparameters
        #self.net = Model()
        #self.net.load_state_dict(torch.load(os.path.join(args.model_dir,'model-23-96000.pth')))
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([3]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([3])),
            batch_shape=torch.Size([3])
        )

    def forward(self, x):
        # The forward function should be written as if we were dealing with each output
        # dimension in batch
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
class GPModel(gpytorch.Module):
    def __init__(self, inducing_points):
        super(GPModel, self).__init__()
        self.gp = MultitaskGPModel(inducing_points)
        self.likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)

    def forward(self, x):
        output = self.gp(x)
        return output

## Creat Model

In [6]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(device)

model = GPModel(torch.zeros(3, args.batch_size, 128)).to(device)
# Load Resnet
'''
state_dict = torch.load(os.path.join('/notebooks/global_localization/dual_resnet_torch','pretrained.pth'))
for name,param in state_dict.items():
    print(name, param.shape)
print('Parameters layer:',len(state_dict.keys()))
model.net.load_state_dict(state_dict)
'''

"\nstate_dict = torch.load(os.path.join('/notebooks/global_localization/dual_resnet_torch','pretrained.pth'))\nfor name,param in state_dict.items():\n    print(name, param.shape)\nprint('Parameters layer:',len(state_dict.keys()))\nmodel.net.load_state_dict(state_dict)\n"

In [7]:

state_dict = torch.load(os.path.join(args.model_dir,'pretrained.pth'))
for name,param in state_dict.items():
    print(name, param.shape)
print('Parameters layer:',len(state_dict.keys()))
model.load_state_dict(state_dict)


gp.variational_strategy.base_variational_strategy.inducing_points torch.Size([3, 300, 128])
gp.variational_strategy.base_variational_strategy.variational_params_initialized torch.Size([])
gp.variational_strategy.base_variational_strategy.updated_strategy torch.Size([])
gp.variational_strategy.base_variational_strategy._variational_distribution.variational_mean torch.Size([3, 300])
gp.variational_strategy.base_variational_strategy._variational_distribution.chol_variational_covar torch.Size([3, 300, 300])
gp.mean_module.constant torch.Size([3, 1])
gp.covar_module.raw_outputscale torch.Size([3])
gp.covar_module.base_kernel.raw_lengthscale torch.Size([3, 1, 1])
likelihood.raw_noise torch.Size([1])
likelihood.noise_covar.raw_noise torch.Size([3])
Parameters layer: 10


<All keys matched successfully>

In [8]:
# Disable net
for param in model.parameters():
    param.requires_grad = False
    
#for param in model.net.parameters():
#    param.requires_grad = False
    
# Display Learn parameters
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.shape)

# Training
## Parameters

In [11]:
#args.norm_mean = args.norm_mean.to(device)
#args.norm_std = args.norm_std.to(device)

optimizer = optim.Adam([
    {'params': model.gp.parameters(), \
     'lr': args.learning_rate,'weight_decay':args.weight_decay},
    {'params': model.likelihood.parameters(), \
     'lr': args.learning_rate,'weight_decay':args.weight_decay},
])
'''
optimizer = optim.Adam([
    {'params': model.gp.parameters(), \
     'lr': args.learning_rate,'weight_decay':args.weight_decay},
    {'params': model.likelihood.parameters(), \
     'lr': args.learning_rate,'weight_decay':args.weight_decay},
])
'''
scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch: args.decay_rate**epoch)
#mll = gpytorch.mlls.VariationalELBO(model.likelihood, model.gp, num_data=len(dataset.Targets))
mll = gpytorch.mlls.PredictiveLogLikelihood(model.likelihood, model.gp, num_data=len(dataset))

In [12]:
#print('CNN model parameters:', sum(param.numel() for param in model.net.global_context.parameters()))
#print('Regressor model parameters:', sum(param.numel() for param in model.net.global_regressor.parameters()))
print('GP model parameters:', sum(param.numel() for param in model.gp.parameters()))
print('Likelihood parameters:', sum(param.numel() for param in model.likelihood.parameters()))

GP model parameters: 386109
Likelihood parameters: 4


## Training Epoch

In [13]:
def train(e):
    train_loss = 0.
    with gpytorch.settings.num_likelihood_samples(100):
        for b, data in enumerate(dataloader, 0):
            start = time.time()
            with torch.no_grad():
                x,y = data
                x,y = x.to(device),y.to(device)
                # normalize targets
                trans_target, _ = torch.split(y, [3, 4], dim=1)
                
            optimizer.zero_grad()
            output = model(x)
            trans_loss = -mll(output, trans_target)
            #rot_loss = 1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_target),dim=1)))
            total_loss = trans_loss #+ args.lamda_weights * rot_loss
            total_loss.backward()
            optimizer.step()
            
            end = time.time()
            with torch.no_grad():
                train_loss += float(total_loss)
                lr = scheduler.get_last_lr()[0]
                writer.add_scalars('training loss',
                  {'item loss':float(total_loss),
                  'batch loss':train_loss/(b+1)},
                  e * len(dataloader) + (b+1))
                if ((b+1)%args.display == 0):
                     print(
                        "{}/{} (epoch {}), train_loss = {}, time/batch = {:.3f}, learning rate = {:.9f}"
                        .format(
                        e * len(dataloader) + (b+1),
                        args.num_epochs * len(dataloader),
                        e,
                        train_loss/(b+1),
                        end - start,
                        lr)) # scheduler.get_last_lr()[0]
                if (e * len(dataloader) + (b+1)) % args.save_every == 0:
                    checkpoint_path = os.path.join(args.model_dir, 'model-{}-{}.pth'.format(e, e * len(dataloader) + (b+1)))
                    torch.save(model.state_dict(),checkpoint_path)
                    print('saving model to model-{}-{}.pth'.format(e, e * len(dataloader) + (b+1)))

In [14]:
model.gp.train()
model.likelihood.train()

MultitaskGaussianLikelihood(
  (noise_covar): MultitaskHomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
  (raw_noise_constraint): GreaterThan(1.000E-04)
)

In [15]:
for e in range(args.num_epochs):
#for e in range(10):
    train(e)
    scheduler.step()

50/10400 (epoch 0), train_loss = -6.708509473800659, time/batch = 0.029, learning rate = 0.005000000
102/10400 (epoch 1), train_loss = -7.041602039337159, time/batch = 0.029, learning rate = 0.004900000
154/10400 (epoch 2), train_loss = -7.036065826416015, time/batch = 0.029, learning rate = 0.004802000
206/10400 (epoch 3), train_loss = -7.060898513793945, time/batch = 0.029, learning rate = 0.004705960
258/10400 (epoch 4), train_loss = -7.047835235595703, time/batch = 0.029, learning rate = 0.004611841
310/10400 (epoch 5), train_loss = -7.034045782089233, time/batch = 0.029, learning rate = 0.004519604
362/10400 (epoch 6), train_loss = -7.049473524093628, time/batch = 0.030, learning rate = 0.004429212
414/10400 (epoch 7), train_loss = -7.051306943893433, time/batch = 0.030, learning rate = 0.004340628
466/10400 (epoch 8), train_loss = -7.065126686096192, time/batch = 0.029, learning rate = 0.004253815
518/10400 (epoch 9), train_loss = -7.069210090637207, time/batch = 0.029, learning 

4158/10400 (epoch 79), train_loss = -7.14430950164795, time/batch = 0.029, learning rate = 0.001013515
4210/10400 (epoch 80), train_loss = -7.15402325630188, time/batch = 0.029, learning rate = 0.000993244
4262/10400 (epoch 81), train_loss = -7.153731698989868, time/batch = 0.029, learning rate = 0.000973379
4314/10400 (epoch 82), train_loss = -7.1505357837677, time/batch = 0.029, learning rate = 0.000953912
4366/10400 (epoch 83), train_loss = -7.144216890335083, time/batch = 0.029, learning rate = 0.000934834
4418/10400 (epoch 84), train_loss = -7.152528467178345, time/batch = 0.029, learning rate = 0.000916137
4470/10400 (epoch 85), train_loss = -7.156867475509643, time/batch = 0.029, learning rate = 0.000897814
4522/10400 (epoch 86), train_loss = -7.1508048629760745, time/batch = 0.029, learning rate = 0.000879858
4574/10400 (epoch 87), train_loss = -7.156377058029175, time/batch = 0.029, learning rate = 0.000862261
4626/10400 (epoch 88), train_loss = -7.155984210968017, time/batch 

8214/10400 (epoch 157), train_loss = -7.179686641693115, time/batch = 0.030, learning rate = 0.000209635
8266/10400 (epoch 158), train_loss = -7.178277568817139, time/batch = 0.030, learning rate = 0.000205442
8318/10400 (epoch 159), train_loss = -7.179118385314942, time/batch = 0.029, learning rate = 0.000201333
8370/10400 (epoch 160), train_loss = -7.176823396682739, time/batch = 0.029, learning rate = 0.000197307
8422/10400 (epoch 161), train_loss = -7.179263410568237, time/batch = 0.029, learning rate = 0.000193361
8474/10400 (epoch 162), train_loss = -7.179762773513794, time/batch = 0.029, learning rate = 0.000189493
8526/10400 (epoch 163), train_loss = -7.182591714859009, time/batch = 0.030, learning rate = 0.000185704
8578/10400 (epoch 164), train_loss = -7.182411890029908, time/batch = 0.029, learning rate = 0.000181990
8630/10400 (epoch 165), train_loss = -7.177892236709595, time/batch = 0.029, learning rate = 0.000178350
8682/10400 (epoch 166), train_loss = -7.183656597137451

# Evaluate

In [21]:
def denormalize_navie(normed_target, norm_mean, norm_std):
    target_trans_unscaled = normed_target * norm_std
    target_trans_uncentered = target_trans_unscaled + norm_mean
    
    return target_trans_uncentered

args.norm_mean = torch.from_numpy(
    np.genfromtxt('/notebooks/michigan_nn_data/toy/norm_mean.csv', delimiter=',')
    ).float()
args.norm_std = torch.from_numpy(
    np.genfromtxt('/notebooks/michigan_nn_data/toy/norm_std.csv', delimiter=',')
    ).float()
args.norm_mean = args.norm_mean.to(device)
args.norm_std = args.norm_std.to(device)

model.eval()
loss = 0
for b, data in enumerate(dataloader, 0):
    start = time.time()
    with torch.no_grad():
        x,y = data
        x,y = x.to(device),y.to(device)
        # normalize targets
        trans_target, _ = torch.split(y, [3, 4], dim=1)
        output = model(x)
        
        trans_pred = denormalize_navie(output.mean,args.norm_mean,args.norm_std).cpu().numpy()
        trans_gt = denormalize_navie(y[:,:3],args.norm_mean,args.norm_std).cpu().numpy()
        
        #batch_loss = np.sum(np.abs((trans_pred - trans_gt)),axis = 0)
        batch_loss = np.sqrt(np.sum((trans_pred - trans_gt)**2,axis=1))
        if b == 0:
            loss = batch_loss
        else:
            loss = np.hstack((loss,batch_loss))
#loss = loss/len(dataset)
loss

array([1.4118478, 3.1710045, 2.858386 , ..., 4.6196055, 1.0426774,
       2.1239047], dtype=float32)

In [27]:
np.mean(loss)

4.3217773

In [26]:
np.median(loss)

3.4699101

In [28]:
x_test = torch.from_numpy(np.genfromtxt('/notebooks/michigan_nn_data/toy/x_test.csv', delimiter=',')).float()
y_test = torch.from_numpy(np.genfromtxt('/notebooks/michigan_nn_data/toy/y_test.csv', delimiter=',')).float()
dataset_test = TensorDataset(x_test,y_test)

#[args.norm_mean, args.norm_std] = [torch.tensor(x) for x in dataset.get_norm()]
#torch.save([args.norm_mean, args.norm_std], '/notebooks/global_localization/norm_mean_std.pt')

dataloader_test = DataLoader(dataset_test, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=True, pin_memory=True)


model.eval()
loss = 0
for b, data in enumerate(dataloader_test, 0):
    start = time.time()
    with torch.no_grad():
        x,y = data
        x,y = x.to(device),y.to(device)
        # normalize targets
        trans_target, _ = torch.split(y, [3, 4], dim=1)
        output = model(x)
        
        trans_pred = denormalize_navie(output.mean,args.norm_mean,args.norm_std).cpu().numpy()
        trans_gt = denormalize_navie(y[:,:3],args.norm_mean,args.norm_std).cpu().numpy()
        
        #batch_loss = np.sum(np.abs((trans_pred - trans_gt)),axis = 0)
        batch_loss = np.sqrt(np.sum((trans_pred - trans_gt)**2,axis=1))
        if b == 0:
            loss = batch_loss
        else:
            loss = np.hstack((loss,batch_loss))
#loss = loss/len(dataset)
loss

array([12.728456 ,  4.7990246,  1.2676868, ...,  7.4782877,  3.5493639,
        7.1898255], dtype=float32)

In [29]:
np.mean(loss)

4.408299

In [30]:
np.median(loss)

3.5619557